# **5310 Organizations**

### Looking into 5310 Organizations with the following project types:
*  purchasing wheelchair accessible vehicle
* on-demand scheduling 
* dispatching software platforms

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
from siuba import *
import numpy as np

import altair as alt

from shared_utils import altair_utils
from shared_utils import geography_utils
from shared_utils import calitp_color_palette as cp
from shared_utils import styleguide

from dla_utils import _dla_utils

from calitp import to_snakecase

from IPython.display import display, Markdown, HTML, display_html

In [3]:
df = pd.read_csv('5310_info_pt.csv')

In [4]:
grouped =pd.read_csv('gs://calitp-analytics-data/data-analyses/5310/5310_org_info_manualfill.csv')

In [5]:
org_counts = grouped >> count(_.organization_type)
_dla_utils.bar_chart_nosubset(
    org_counts, "organization_type", "n", "organization_type", "5310 Organization Types"
)

alt.Chart(...)

In [6]:
single_county = df[~df["county"].str.contains(",", na=False)]
_dla_utils.bar_chart_nosubset((single_county>>count(_.county)),
    "county",
    "n",
    "county",
    "Counties with 5310 Organizations (Single County)"
)

alt.Chart(...)

In [7]:
county_explode = df>>select(_.organization_name, _.ntd_id, _.itp_id, _.organization_type, _.city, _.county, _.merge_status)
county_explode.explode('county')

county_explode2 = county_explode.assign(county_split = df['county'].str.split(', ')).drop("county", axis=1)\
    .rename(columns = {"county_split": "county"})
county_explode2 = county_explode2.explode('county')

_dla_utils.bar_chart_nosubset((county_explode2>>count(_.county)),
    "county",
    "n",
    "county",
    "Counties with 5310 Organizations Operating"
)

alt.Chart(...)

## Project Types

In [8]:
#df.sample()

In [9]:
ptype_counts = (df
 >>group_by(_.organization_name)
 >>summarize(n_vp = _.vehicle_purchase.sum(), 
            n_oa = _.operating_assistance.sum(),
            n_mm = _.mobility_management.sum(),
            n_hsp = _.hardware_software_purchase.sum(),
            n_c = _.communications.sum(),
            n_fe = _.facilities_eqp.sum(),
             n_s = _.surveillance.sum(),
             n_sub = _.subsidy.sum()
            )
)

In [10]:
# columns = ['n_vp','n_oa','n_mm','n_hsp','n_c','n_fe','n_s','n_sub']

# for column in columns: 
#     ptype_counts.column[ptype_counts[column]> 0] = 1

#WORKED for getting yes/no present
## from https://stackoverflow.com/questions/61996932/replacing-values-greater-1-in-a-large-pandas-dataframe

#ptype_counts.set_index('organization_name',inplace=True)
#ptype_counts[ptype_counts >= 1] = 'Yes'
#ptype_counts.reset_index()

#ptype_counts = ptype_counts.replace(0, 'No')

#ptype_counts.to_csv('single_counts_2.csv')

# # def project_present(df):
# #     columns = ['n_vp','n_oa','n_mm','n_hsp','n_c','n_fe','n_s','n_sub']
# #     if df.column != 0:
# #         return 1
# #     else:
# #         return 0

# # for column in columns:
# #     df[f"{column}_2"] = df.apply(lambda x: project_present(x), axis=1)
    
    
    
# columns = ['n_vp','n_oa','n_mm','n_hsp','n_c','n_fe','n_s','n_sub']

# for column in columns:
#     ptype_counts[f"{column}_2"] = ptype_counts[column].any(1).astype(int)

In [11]:
ptype_counts.set_index('organization_name',inplace=True)
ptype_counts[ptype_counts >= 1] = 1
ptype_counts = ptype_counts.reset_index()

In [12]:
sum_ptype = ((ptype_counts).rename(columns = _dla_utils.labeling)
               .agg({'Vehicle Purchase':'sum', 'Operating Assistance':'sum',
                      'Mobility Management':'sum','Hardware/Software Purchase':'sum','Communications':'sum',
                     'Facilities Equipment':'sum', 'Surveillance':'sum', 'Subsidies':'sum'}).reset_index())
sum_ptype = sum_ptype.rename(columns = {"index":"Project Types",0:"Count"})
_dla_utils.bar_chart_nosubset((sum_ptype),
    "Project Types",
    "Count",
    "Project Types",
    "Number of Organizations using Project Types"
)

alt.Chart(...)

In [13]:
columns = [
        "n_vp",
        "n_oa",
        "n_mm",
        "n_hsp",
        "n_c",
        "n_fe",
        "n_s",
        "n_sub"]

for col in columns:
    org_counts = (ptype_counts 
                  >> filter(_[col] == 1)
                  >> count(_['organization_name'])
                  >> select(_['organization_name']))
    table = (_dla_utils.pretify_tables2(org_counts, "Organization Name", ""))

#     display(HTML(f"<br>"
#                  f"<br>"
#                  f"<strong>Organization with {column} Project Types Present</strong>"))
    
#     (display(HTML(_dla_utils.pretify_tables2(org_counts, "Organization Name", ""))))

In [14]:
#display(HTML(table))

In [15]:
cols = [
        "n_vp",
        "n_oa",
        "n_mm",
        "n_hsp",
        "n_c",
        "n_fe",
        "n_s",
        "n_sub"]
d = {}
for i in cols:
    org_counts = ((ptype_counts 
                  >> filter(_[i] == 1)
                  >> count(_['organization_name'])
                  >> select(_['organization_name'])
                  )
## Commented out portion works to format df but does not work with the side by side               
                 #  .rename(columns={'organization_name':f"Organizations in {_dla_utils.labeling(i)}"})
                  )

    # d[str(i)] = (org_counts
    #               .style.hide(axis='index')
    #               .set_properties(**{'text-align': 'center'})
    #               .set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
    #              )
    d[str(i)] = ((_dla_utils.pretify_tables2(org_counts, "Organization Name", "")))
    


In [16]:
table1 = d['n_vp']
table2 = d['n_oa']
table3 = d['n_mm']
table4 = d['n_hsp']
table5 = d['n_c']
table6 = d['n_fe']
table7 = d['n_s']
table8 = d['n_sub']

In [17]:
#table = display(HTML(table6))

In [18]:
def pretify_tables3(df):
    df = df.rename(columns = _dla_utils.labeling)
    #display(HTML(df.to_html(index=False)))
    df = (df.style.hide(axis='index')
              .set_properties(**{'text-align': 'center'})
              .set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
             )
    return (df.to_html(index=False))

In [19]:
def display_side_by_side(*args):
        html_str=''
        for df in args:
            html_str+=df
        display_html(html_str.replace('table','table style="display:inline"'),raw=True)

        

In [20]:
cols = [
        "n_vp",
        "n_oa",
        "n_mm",
        "n_hsp",
        "n_c",
        "n_fe",
        "n_s",
        "n_sub"]
d2 = {}
for i in cols:
    org_counts = ((ptype_counts 
                  >> filter(_[i] == 1)
                  >> count(_['organization_name'])
                  >> select(_['organization_name'])
                  )
## Commented out portion works to format df but does not work with the side by side               
                  .rename(columns={'organization_name':f"Organizations with {_dla_utils.labeling(i)} Project Types"})
                  )

    # d2[str(i)] = (org_counts
    #               .style
    #               .hide(axis='index')             
    #               .set_properties(**{'text-align': 'center'})
    #               .set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
    #              )
    d2[str(i)] = ((pretify_tables3(org_counts))
                  # .set_properties(**{'text-align': 'center'})                 ##doesnt work: use fx above
                  # .set_table_styles([dict(selector='th', props=[('text-align', 'center')])])   ##doesnt work: use fx above
                 )
    


In [21]:
table11 = d2['n_vp']
table22 = d2['n_oa']
table33 = d2['n_mm']
table44 = d2['n_hsp']
table55 = d2['n_c']
table66 = d2['n_fe']
table77 = d2['n_s']
table88 = d2['n_sub']

In [22]:
display(HTML(table66))

Organizations With Facilities Equipment Project Types
Eastern Sierra Transit Authority
Lighthouse for the Blind and Visually Impaired
Milestones of Development INC
North and South of Market Adult Day Health
Porterville Sheltered Workshop


In [23]:
display_side_by_side(table11, table22)

Organizations With Vehicle Purchase Project Types
ARC Bakersfield
ARC Imperial Valley
Able Industries
Alegria Community Living
Angel View Inc
Asian Community Center of Sacramento Valley Inc DBA ACC Senior Services
Avenidas
Bay Area Outreach & Recreation Program Inc
Big Valley 50 Plus
Butte County Association of Governments Butte Regional Transit


In [24]:
display_side_by_side(table33, table44)

Organizations With Mobility Management Project Types
Amador Transit
Catholic Charities of the Diocese of Stockton
City of Roseville
City of Visalia
Common Ground Senior Services
Contra Costa Transportation Authority (CCTA)
County of Sonoma Human Services Department Adult and Aging Division
Eden I&R INC
Friends of Children with Special Needs
Full Access & Coordinated Transportation Inc


In [25]:
display_side_by_side(table55, table66)

Organizations With Communications Project Types
Center for Elders' Independence
Community Bridges Liftline
Eastern Sierra Transit Authority
Humboldt Senior Resource Center
North of the River Recreation and Park District
Porterville Sheltered Workshop
Self-Help for the Elderly
United Cerebral Palsy Association of Greater Sacramento Inc
Work Training Center
Organizations With Facilities Equipment Project Types


In [26]:
display_side_by_side(table77, table88)

Organizations With Surveillance Project Types
Community Bridges Liftline
Institute on Aging
Porterville Sheltered Workshop
United Cerebral Palsy Association of Greater Sacramento Inc
Organizations With Subsidies Project Types
San Mateo County Transit District


In [27]:
# def display_side_by_side2(*args,titles=cycle([''])):
#     html_str=''
#     for df,title in zip(args, chain(titles,cycle)):
#         html_str+=df
#         html_str+=f'<h2>{title}</h2>'
#         display_html(html_str.replace('table','table style="display:inline"'),raw=True)

In [28]:
#from itertools import chain,cycle

# def display_side_by_side2(*args,titles=cycle([''])):
#     html_str=''
#     for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
#         html_str+='<th style="text-align:center"><td style="vertical-align:top">'
#         html_str+=f'<h2>{title}</h2>'
#         html_str+=df.to_html().replace('table','table style="display:inline"')
#         html_str+='</td></th>'
#     display_html(html_str,raw=True)

In [29]:
# display_side_by_side(table1, table2
#                   #   titles=["Foo", "Foo Bar"]
#                      )

In [30]:
#display_side_by_side2(table1, table2, ['Vehicle Purchase','Operating Assistance']) 

### Purchasing Wheelchair Accessible Vehicles

In [31]:
display(
    HTML(f"Of the {len(df>>filter(_.vehicle_purchase==1)>>count(_.organization_name))} agencies using 5310 funds "
      f"for vehicle purchases, "
      f"{len(df>>filter(_.vehicle_purchase==1)>>filter(_.description.str.contains('Bus'))>>count(_.organization_name))} "
      f"are for Bus Purchases, "
      f"and {len(df>>filter(_.vehicle_purchase==1)>>filter(_.description.str.contains('Van'))>>count(_.organization_name))} "
      f"are for Van Purchases"
         f"<br> There are {len(df>>filter(_.description.str.contains('wheelchair')))} specific mentions of wheelchair "
         f"accessible vehicles, ramps or other accessiblity features."
        ))

**Organizations using funds for Vehicle Purchase**

In [32]:
vehicle_purchase= (df>>filter(_.vehicle_purchase==1)>>count(_.organization_name)>>arrange(-_.n)).head(10)

display(HTML(_dla_utils.pretify_tables2(vehicle_purchase, "", "Count")))


Organization Name,Count
On Lok Senior Health Services,50
Pride Industries One Inc,49
United Cerebral Palsy of San Luis Obispo County,46
United Cerebral Palsy Association of Greater Sacramento Inc,42
Kings Rehabilitation Center,30
Monterey-Salinas Transit,26
North of the River Recreation and Park District,25
ARC Bakersfield,20
Vivalon Inc,20
Butte County Association of Governments Butte Regional Transit,19


### On-demand Scheduling

**Organizations with Operating Assistance Project types**

In [33]:
display(HTML(f"There is {len(df>>filter(_.description.str.contains('demand')))} mention of Demand related project types"))
display(HTML(f"There are {len((df>>filter(_.operating_assistance==1)>>count(_.organization_name)>>arrange(-_.n)))} "
             f"mentions of Operating Assistance"))

In [34]:
operating_assistance= (df>>filter(_.operating_assistance==1)>>count(_.organization_name)>>arrange(-_.n)).head(10)

display(HTML(_dla_utils.pretify_tables2(operating_assistance, "", "Count")))

Organization Name,Count
Independent Living Partnership,8
Asian Community Center of Sacramento Valley Inc DBA ACC Senior Services,6
ARC Imperial Valley,4
Choice in Aging,4
County of Sonoma Human Services Department Adult and Aging Division,4
Mobility Matters,4
Angel View Inc,3
Catholic Charities of the Diocese of Stockton,3
City of Lafayette: Lamorinda Sprit Van Program,3
County of Sacramento Department of Child Family and Adult Services,3


### Dispatching Software Platforms

In [35]:
display(
    HTML(f"Out of the {len(df>>filter(_.hardware_software_purchase==1)>>count(_.organization_name))} "
         f"with harware or software purchases, "
         f"{len(df>>filter(_.hardware_software_purchase==1)>>filter(_.description.str.contains('Software'))>>count(_.organization_name))}" 
             f"organization used 5310 funds for software purposes"))
             

In [36]:
software = (
    df
    >> filter(_.hardware_software_purchase == 1)
    >> filter(_.description.str.contains("Software"))
    >> count(_.organization_name)
)
display(HTML(_dla_utils.pretify_tables2(software, "", "Count")))

Organization Name,Count
Asian Community Center of Sacramento Valley Inc DBA ACC Senior Services,2
Catholic Charities of the Diocese of Stockton,2
County of Sonoma Human Services Department Adult and Aging Division,1
Yolo County Transportation District,2


## Project Type Funding Analysis

### By Organization & Year

**Sum of Funds Allocated to Organizations by Year (Top 20)**

In [37]:
top20_sum = (
    df
    >> group_by(_.organization_name, _.project_year)
    >> summarize(sum_allocated=_.allocationamount.sum())
    >> arrange(-_.sum_allocated)
).head(20)

top20_sum.sum_allocated = top20_sum.sum_allocated.map('${:,.2f}'.format)

display(HTML(_dla_utils.pretify_tables2(top20_sum,"", "Sum Allocated")))

Organization Name,Project Year,Sum Allocated
United Cerebral Palsy Association of Greater Sacramento Inc,2017,"$1,782,550.00"
On Lok Senior Health Services,2019,"$1,480,000.00"
Pride Industries One Inc,2019,"$1,257,714.00"
Asian Community Center of Sacramento Valley Inc DBA ACC Senior Services,2017,"$1,246,128.00"
Mobility Matters,2017,"$1,160,438.00"
Friends of Children with Special Needs,2019,"$1,152,937.00"
Pride Industries One Inc,2017,"$1,135,727.52"
Independent Living Partnership,2017,"$1,114,031.00"
ARC Bakersfield,2017,"$1,008,340.28"
Marin County Transit District,2017,"$1,000,000.00"


### By Organization & Project Types

**Vehicle Purchases**

In [38]:
vp = (df
 >>filter(_.vehicle_purchase==1)
 >> group_by(_.organization_name, _.project_year)
 >> summarize(sum_allocated = _.allocationamount.sum())
  >> arrange(-_.sum_allocated)
).head(20)

# vp = (vp.style.hide(axis='index').format(formatter={("sum_allocated"): "${:,.2f}"}))
# display(vp)
vp.sum_allocated = vp.sum_allocated.map('${:,.2f}'.format)
display(HTML(_dla_utils.pretify_tables2(vp,"", "Sum Allocated")))

Organization Name,Project Year,Sum Allocated
On Lok Senior Health Services,2019,"$1,480,000.00"
United Cerebral Palsy Association of Greater Sacramento Inc,2017,"$1,449,680.00"
Pride Industries One Inc,2019,"$1,209,000.00"
Pride Industries One Inc,2017,"$1,135,727.52"
ARC Bakersfield,2017,"$1,008,340.28"
Monterey-Salinas Transit,2017,"$980,000.00"
United Cerebral Palsy of San Luis Obispo County,2017,"$634,000.00"
Asian Community Center of Sacramento Valley Inc DBA ACC Senior Services,2017,"$619,600.00"
El Dorado County Transit Authority,2017,"$610,000.00"
Institute on Aging,2017,"$560,000.00"


**Operating Assistance**

In [39]:
oa= (df
 >>filter(_.operating_assistance==1)
 >> group_by(_.organization_name, _.project_year)
 >> summarize(sum_allocated = _.allocationamount.sum())
 >> arrange(-_.sum_allocated)
).head(20)


oa.sum_allocated = oa.sum_allocated.map('${:,.2f}'.format)
display(HTML(_dla_utils.pretify_tables2(oa,"", "Sum Allocated")))

Organization Name,Project Year,Sum Allocated
Independent Living Partnership,2017,"$1,114,031.00"
LIFE ElderCare,2017,"$802,658.00"
Catholic Charities of the Diocese of Stockton,2017,"$721,986.00"
Town of Truckee,2017,"$702,506.00"
Marin County Transit District,2017,"$700,000.00"
West Valley Community Services,2019,"$649,453.00"
Asian Community Center of Sacramento Valley Inc DBA ACC Senior Services,2017,"$624,048.00"
Dignity Health Connected Living,2017,"$530,302.00"
Community Bridges Liftline,2017,"$475,200.00"
United Cerebral Palsy Association of Greater Sacramento Inc,2019,"$467,625.00"


**Software Purchase**

In [40]:
sp = (df
 >>filter(_.hardware_software_purchase==1)
 >>filter(_.description.str.contains('Software'))
 >> group_by(_.organization_name, _.project_year)
 >> summarize(sum_allocated = _.allocationamount.sum())
  >> arrange(-_.sum_allocated)
).head(20)

sp.sum_allocated = sp.sum_allocated.map('${:,.2f}'.format)
display(HTML(_dla_utils.pretify_tables2(sp, "", "Sum Allocated")))

Organization Name,Project Year,Sum Allocated
Catholic Charities of the Diocese of Stockton,2019,"$20,000.00"
County of Sonoma Human Services Department Adult and Aging Division,2018,"$10,000.00"
Asian Community Center of Sacramento Valley Inc DBA ACC Senior Services,2017,$0.00
Yolo County Transportation District,2017,$0.00


In [41]:
hsp = (df
 >>filter(_.hardware_software_purchase==1)
 >> group_by(_.organization_name, _.project_year)
 >> summarize(sum_allocated = _.allocationamount.sum())
       >> arrange(-_.sum_allocated)
).head(20)

hsp.sum_allocated = hsp.sum_allocated.map('${:,.2f}'.format)
display(HTML(_dla_utils.pretify_tables2(hsp, "", "Sum Allocated")))

Organization Name,Project Year,Sum Allocated
United Cerebral Palsy Association of Greater Sacramento Inc,2017,"$61,600.00"
Catholic Charities of the Diocese of Stockton,2019,"$28,500.00"
Community Bridges Liftline,2017,"$17,196.00"
County of Sonoma Human Services Department Adult and Aging Division,2018,"$10,000.00"
Community Bridges Liftline,2019,"$7,954.00"
Porterville Sheltered Workshop,2019,"$7,742.00"
San Joaquin Regional Transit District,2017,"$7,571.00"
Sunline Transit Agency,2019,"$5,150.00"
Asian Community Center of Sacramento Valley Inc DBA ACC Senior Services,2017,"$2,480.00"
Milestones of Development INC,2019,"$1,400.00"


### By County

In [42]:
county = (df
 >> group_by(_.county)
 >> summarize(sum_allocated = _.allocationamount.sum())
)

county.sum_allocated = county.sum_allocated.map('${:,.2f}'.format)
display(HTML(_dla_utils.pretify_tables2(county, "County", "Sum Allocated")))

County,Sum Allocated
Alameda,"$2,982,664.00"
"Alameda, Contra Costa","$671,900.00"
Amador,"$469,995.00"
"Amador, Calaveras","$299,479.40"
Butte,"$1,235,790.10"
"Butte, Glen","$813,194.00"
Calaveras,$0.00
"Calaveras, Amador, Tuolumne","$826,157.00"
Colusa,"$162,400.00"
Contra Costa,"$3,007,198.00"
